In [1]:
ACCESS_TOKEN = "1213270339293077504-0VGZgQTrbUs3cNfITH8SkytxGuLmcE"
ACCESS_TOKEN_SECRET = "zPqKpQumQkcKf52iIakDtjH6C6kbFN6M4w2H4im1yZ48a"
CONSUMER_KEY = "GgMgpR0X0Bf3j6Y8UzeBY5eCp"
CONSUMER_SECRET = "hA13Bslmb4Bi9Vo4WGwRKSVELiksYmGGOiH7lITvWnarS4HOMn"

In [ ]:
import tweepy
from datetime import datetime
import csv
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import json

keywords = ['#Bernie', '#Trump', '#petebuttigieg']
#keywords = ['#Democrats2020','#]
filename = 'C:\\Users\\abhin\\Downloads\\twitter_tweets.json'
end_date = datetime.strptime("2020-02-16", '%Y-%m-%d').date()
flag = False
twitter_data = {}

class stdOutListener(StreamListener):
    
    def __init__(self, filename):
        self.filename = filename
        
    def on_data(self, stream):
        
        data = json.loads(stream)
        tweet_date = datetime.strptime(data['created_at'], '%a %b %d %H:%M:%S %z %Y').date()
        
        if 'retweeted_status' in [*data]:
            print("Retweet found, ignoring!")
            return
    
        if(tweet_date < end_date):
            return False
        
        f = open('tweets.json', 'w'):
            json.dump(data, f)
            
        print(data['text'])
        
    def on_error(self, status_code):
        if status_code == 420:
            return False
        
class twitterStreamer():
    
    def setConnection(self, filename, keywords):
        try:
            listener = stdOutListener(filename)
            auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
            auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
            print("Connection has been established successfully!")
            return(auth, listener)
        
        except:
            print("Connection not established. Try Again Later.")
            
    def getTweets(self, auth, listener):
        stream = Stream(auth, listener)
        stream.filter(track=keywords)
    
if __name__ == "__main__":
    tweets = twitterStreamer()
    auth, listener = tweets.setConnection(filename, keywords) 
    tweets.getTweets(auth, listener)
    print("HERE!!!!!!")

Connection has been established successfully!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
As other candidates go negative and dive into mudslinging, we have a positive campaign in our grasp!

If we get beh… https://t.co/Df1OdYfz90
Yes, what the Democrats did to Tulsi in 2020 might be worse  than what the Republicans did to Ron Paul in 2012.… https://t.co/04oGqXiZzj
Retweet found, ignoring!
Retweet found, ignoring!
@realDonaldTrump
Already broke promises to American workers by supporting LEGAL #INDIA visas.
Charlotte, NC is… https://t.co/7ZoUm4y7cP
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
The truth hurts. Too bad for the orange blimp.
Retweet found, ign

Retweet found, ignoring!
Retweet found, ignoring!
Oh wow!  ❤❤
Retweet found, ignoring!
@Jamie_Margolin Seeing @BernieSanders with @lsarsour the #Terrorist makes me think #Bernie isn't for me.… https://t.co/EfSZgbqHbW
Retweet found, ignoring!
Retweet found, ignoring!
#Trump yönetimi Çin'e #teknoloji ürünleri ihracatına yeni kısıtlamalar planlıyor. 

Bu dalgada #Çin'in yolcu uçağı… https://t.co/bH7T4Lxxnp
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️⬇️
Retweet found, ignoring!
@carlottadale38 @magi_jay It reminds me of when Omarosa said a very similar thing about #Trump, how those who oppos… https://t.co/GjnwO5TByM
Retweet found, ignoring!
Since you three @rep_stevewomack @JohnBoozman @SenTomCotton voted to keep #trump in office, When will NWA, not you… https://t.co/CE8BK1gxIv
How many former DOJ Officials are there eligible to sign this letter?

How many will sign?

What percentage?

I thi… https://t.co/tl6JxY3LOK
#trump seems upset th

Retweet found, ignoring!
Retweet found, ignoring!
@realDonaldTrump Pres #Trump pass law req #judge to order lifetime #chemical #castration for #men released on… https://t.co/xwNp7azk5f
@Jim_Jordan @realDonaldTrump OK #PedophileProtector

Lets handle whistleblowers like they do in the Ukraine or Russ… https://t.co/xf4amL9H7O
Retweet found, ignoring!
Vote &amp; share! #Bernie
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
@CNNSitRoom @RedStateBlues_ Barr has not only put his thumb on the scale in criminal cases involving friends and as… https://t.co/AUv4V8jgN7
@beemji
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
Retweet found, ignoring!
We should call you Oblahma !!
You really could talk great .
#Obama #blahblahblah.... 
#Trump #Trump2020… https://t.co/qwIo1MeND2
When someone says “I

In [39]:
x = {'a':[1,2,3],'b':[2,3,4]}


In [40]:
if x.keys() == ['a']:
    print(True)

In [36]:
list(x.keys())

['a']

In [42]:
if 'a' in [*x]:
    print(True)

True
